In [9]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [8]:
client = dataiku.api_client()
#prj= client.get_project(dataiku.default_project_key())
#datasets = prj.list_datasets()

#client = DSSClient(host, apiKey)
dss_projects = client.list_project_keys()
print(dss_projects)

['ADMIN_PROJECT', 'BACKEND_BREAKER', 'BASKETBALLv2', 'BASKETBALLv2_2', 'CUSTOMER_CHURN_BROKEN', 'DKU_TSHIRTS', 'DKU_TUTORIAL_BASICS_101', 'GROUP_A_PROJECT', 'GROUP_B_PROJECT', 'THEBIGSLEEP']


In [25]:
for ProjectKey in dss_projects:
    project = client.get_project(ProjectKey)
    datasets = project.list_datasets()
    print("Datasets in Project: ", ProjectKey)
    index = 0
    for dataset in datasets:
#         print (dataset.type)
         if (dataset.type == 'PostgreSQL'):
#        if dataset.type == 'Snowflake':
            print(dataset.name)

            dataset_to_update = prj.get_dataset(dataset.name)
            # Get dataset settings
            settings = dataset_to_update.get_settings()
            # get raw dataset settings, which is editable
            settings_raw = settings.get_raw()
            # pp.pprint(settings_raw['params'])
            if 'schema' in settings_raw['params']:
                if settings_raw['params']['schema'] in ['AWB_CDW','AWB_CDL', 'AWB_ADHOC','AWB_DATAIKU_CDW', 'AWB_DATAIKU_CDL','AWB_DATAIKU_ADHOC']:
                    new_values = workflow_Alteration_df[workflow_Alteration_df['Existing_Source'.strip()] == settings_raw['params']['table']]
                    update_check = new_values.empty
                    if not new_values.empty:
                        print("Inside If", update_check, new_values,"*****", new_values.index[0])
                        settings_raw['params']['catalog'] = new_values['GB_Database'.strip()][new_values.index[0]]
                        settings_raw['params']['schema'] = new_values['GB_Schema'.strip()][new_values.index[0]]
                        settings_raw['params']['table'] = new_values['GB_TBL_Name'.strip()][new_values.index[0]]
                        settings.save()
                        settings_raw["schema"] = {"columns":[]}
                        settings.save()
                        settings = dataset_to_update.autodetect_settings()
                        settings.save()
                        new_values = None
                        index += 1            

Datasets in Project:  ADMIN_PROJECT
Datasets in Project:  BACKEND_BREAKER
Datasets in Project:  BASKETBALLv2
Basketball_Raw_Data_copy


DataikuException: com.dataiku.dip.server.controllers.NotFoundException: dataset does not exist: ADMIN_PROJECT.Basketball_Raw_Data_copy

In [10]:
index = 0
for dataset in datasets:
    #pp.pprint(dataset)
    print(dataset)
    if dataset.type == 'Snowflake':
        dataset_to_update = prj.get_dataset(dataset.name)
        # Get dataset settings
        settings = dataset_to_update.get_settings()
        # get raw dataset settings, which is editable
        settings_raw = settings.get_raw()
        # pp.pprint(settings_raw['params'])
        if 'schema' in settings_raw['params']:
            if settings_raw['params']['schema'] in ['AWB_CDW','AWB_CDL', 'AWB_ADHOC','AWB_DATAIKU_CDW', 'AWB_DATAIKU_CDL','AWB_DATAIKU_ADHOC']:
                new_values = workflow_Alteration_df[workflow_Alteration_df['Existing_Source'.strip()] == settings_raw['params']['table']]
                update_check = new_values.empty
                if not new_values.empty:
                    print("Inside If", update_check, new_values,"*****", new_values.index[0])
                    settings_raw['params']['catalog'] = new_values['GB_Database'.strip()][new_values.index[0]]
                    settings_raw['params']['schema'] = new_values['GB_Schema'.strip()][new_values.index[0]]
                    settings_raw['params']['table'] = new_values['GB_TBL_Name'.strip()][new_values.index[0]]
                    settings.save()
                    settings_raw["schema"] = {"columns":[]}
                    settings.save()
                    settings = dataset_to_update.autodetect_settings()
                    settings.save()
                    new_values = None
                    index += 1